**Toronto Dwellings Analysis Dashboard**
---
---

In [145]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [146]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [147]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

In [148]:
# Import the CSVs to Pandas DataFrames
file_path = Path("C:/Users/DELL/Desktop/w6_Pythonic_Monopoly/resources/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("C:/Users/DELL/Desktop/w6_Pythonic_Monopoly/resources/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

In [149]:
# Getting the data from the top 10 expensive neighbourhoods
df_expensive_neighborhoods = to_data.groupby("neighbourhood").mean()
df_expensive_neighborhoods = df_expensive_neighborhoods.sort_values(by="average_house_value", ascending=False)

df_expensive_neighborhoods_top10= df_expensive_neighborhoods.head(10)
df_expensive_neighborhoods_top10= pd.DataFrame(df_expensive_neighborhoods_top10)

# Calculate the mean number of dwelling types units per year
dwelling_py = pd.DataFrame(to_data.iloc[:,0:8])

# Calculate the average monthly shelter costs for owned and rented dwellings
avg = to_data[["shelter_costs_owned", "shelter_costs_rented"]]

average_m_price = avg.groupby("year").mean()

#
dwelling_py = pd.DataFrame(to_data.iloc[:,0:8])
dwelling_py_sum = dwelling_py.groupby('year').sum()


In [159]:
# Define Panel visualization functions
"""Neighbourhood Map"""
mean_neighborhoods = to_data.groupby("neighbourhood").mean()
mean_neighborhoods.head()
df_neighbourhood_locations.reset_index(drop=True, inplace=True)
mean_neighborhoods.reset_index(drop=True, inplace=True)


df_all_neighbourhoods = pd.concat([df_neighbourhood_locations,mean_neighborhoods],axis = 1)
 
def neighbourhood_map():
    px.set_mapbox_access_token(map_box_api)

    location_map = px.scatter_mapbox(
    df_all_neighbourhoods,
    lat="lat",
    lon="lon",
    size="average_house_value",
    color="average_house_value",
    size_max=15,
    zoom=11,
    title="Averange house values in Toronto")
    return location_map
    
   

In [160]:
def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    df_expensive_neighborhoods_top10_bar= df_expensive_neighborhoods_top10.hvplot.bar(
    "neighbourhood",
    "average_house_value",
    title="Top 10 Expensive Neighbourhoods in Toronto",
    xlabel="Neighbourhood",
    ylabel="average house value",
    height=800,width = 800,
    rot=90)
    return df_expensive_neighborhoods_top10_bar

In [161]:
def number_dwelling_types():
    """Number of dwelling types per year"""
    num_dwelling = dwelling_py.hvplot.bar(
    "year",
    xlabel= "year",
    ylabel="Dwelling Types Unit",
    rot=90,
    groupby="neighbourhood",
    height = 600)
    return num_dwelling

In [162]:
def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    avg_neighbourhood = to_data[["neighbourhood", "average_house_value"]]
    avg_neighbourhood_df = avg_neighbourhood.groupby("year")
    avg_value_neighbourhood = avg_neighbourhood.hvplot.line(
    "year",
    "average_house_value",
    xlabel= "Year",
    ylabel="average house value",
    groupby="neighbourhood")
    return avg_value_neighbourhood

In [163]:
def average_house_value():
    """Average house values per year."""
    avg_hv = to_data[["average_house_value"]]
    avg_hv_df = avg_hv.groupby("year").mean()
    avg_housevalue = avg_hv_df.hvplot(title = "average house value in Toronto")
    return avg_housevalue

In [164]:
def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    line_graph_data =data
    shelter_line_graph = line_graph_data.hvplot.line(title = title, xlabel = xlabel, ylabel = ylabel, color=color)
    return shelter_line_graph


In [165]:
def create_bar_chart(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    
    plot_bar = data
    bar = plot_bar.hvplot.bar(title= title, xlabel = xlabel, ylabel = ylabel, rot=90, color= color, width=600, height = 600)
    return bar
    

In [166]:
# Create a Title for the Dashboard

Title = "## Toronto Dwellings Analysis Dashboard \n"
welcome_msg = " Welcome to Toronto Housing Data's visual repretation"

toronto_housing_map = pn.Column(Title  +welcome_msg,  neighbourhood_map() )

# Create a tab layout for the dashboard
Dwelling_Types = pn.Column("## Dwelling Type", create_bar_chart(dwelling_py_sum.loc[2001],"Dwelling types in Toronto 2001",'2001','Dwelling Type Units',"red"),
                           create_bar_chart(dwelling_py_sum.loc[2006],"Dwelling types in Toronto 2006",'2006','Dwelling Type Units',"blue"),
                           create_bar_chart(dwelling_py_sum.loc[2011],"Dwelling types in Toronto 2011",'2011','Dwelling Type Units',"yellow"),
                           create_bar_chart(dwelling_py_sum.loc[2016],"Dwelling types in Toronto 2016",'2001','Dwelling Type Units',"purple"),
                           number_dwelling_types())

Property_Values = pn.Column(
    "## Property Value Data", top_most_expensive_neighbourhoods(),average_house_value(), average_value_by_neighbourhood()
)

Shelter_Data = pn.Column(
    "## Shelter Data", create_line_chart(average_m_price.iloc[:,0],"average monthly shelter costs for owned dwellings in Toronto","Year","Avg Montly Shelter Costs","blue"),
create_line_chart(average_m_price.iloc[:,1],"average monthly shelter costs for rented dwellings in Toronto","Year","Avg Montly Shelter Costs","yellow"))



# Create the main dashboard


Dashboard = pn.Tabs(
    (
        "Housing Data",
        toronto_housing_map
    ),
    (
        "Dwelling Types",
        Dwelling_Types
    ),
    ("Toronto Neighborhood Analysis",
    Property_Values
    ),
     ("Shelter Data",
    Shelter_Data
    ),
    
)



In [167]:
Dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Bars)
        [3] HoloViews(Bars)
        [4] HoloViews(Bars)
        [5] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                [1] VSpacer()
    [2] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Curve)
        [3] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                [1] VSpacer()
    [3] Column
        [0] Markdown(str)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)